In [1]:
from google.colab import drive
import sys
drive.mount('/content/drive')
project_dir = '/content/drive/MyDrive/Colab Notebooks/project/final/'
sys.path.append(project_dir+'services')

Mounted at /content/drive


In [2]:
%%writefile "/content/drive/MyDrive/Colab Notebooks/project/final/services/user_manager.py"
"""
User manager service for the music recommender app.
Handles user data operations.
"""
import os
import pandas as pd

class UserManager:
    """Manages user data operations."""

    def __init__(self, users_csv_path):
        """
        Initialize the user manager.

        Args:
            users_csv_path (str): Path to the users CSV file
        """
        self.users_csv_path = users_csv_path

    def get_users_df(self):
        """Get the users DataFrame."""
        if os.path.exists(self.users_csv_path):
            return pd.read_csv(self.users_csv_path)
        return pd.DataFrame(columns=[
            'user_id', 'username', 'password', 'first_name', 'last_name',
            'age', 'gender', 'favorite_artists', 'recommended_artists'
        ])

    def save_users_df(self, df):
        """Save the users DataFrame to CSV."""
        df.to_csv(self.users_csv_path, index=False)

    def get_user_by_id(self, user_id):
        """
        Get a user by ID.

        Args:
            user_id (int): The user ID

        Returns:
            pandas.Series or None: The user data if found, otherwise None
        """
        users_df = self.get_users_df()
        user_rows = users_df[users_df['user_id'] == user_id]

        if user_rows.empty:
            return None

        return user_rows.iloc[0]

    def get_user_by_username(self, username):
        """
        Get a user by username.

        Args:
            username (str): The username

        Returns:
            pandas.Series or None: The user data if found, otherwise None
        """
        users_df = self.get_users_df()
        user_rows = users_df[users_df['username'] == username]

        if user_rows.empty:
            return None

        return user_rows.iloc[0]

    def authenticate(self, username, password):
        """
        Authenticate a user.

        Args:
            username (str): The username
            password (str): The password

        Returns:
            tuple: (success, user_id or error_message)
        """
        user = self.get_user_by_username(username)

        if user is None:
            return (False, "Username not found.")

        if user['password'] != password:
            return (False, "Incorrect password.")

        return (True, user['user_id'])

    def create_user(self, username, password, first_name="", last_name="",
                   age=0, gender="", favorite_artists=None):
        """
        Create a new user.

        Args:
            username (str): The username
            password (str): The password
            first_name (str): First name
            last_name (str): Last name
            age (int): Age
            gender (str): Gender
            favorite_artists (list): List of favorite artists

        Returns:
            tuple: (success, user_id or error_message)
        """
        if not username or not password:
            return (False, "Username and password are required.")

        if len(password) < 4:
            return (False, "Password should be at least 4 characters long.")

        users_df = self.get_users_df()

        if username in users_df['username'].values:
            return (False, "Username already taken.")

        favorite_artists_str = ', '.join(favorite_artists) if favorite_artists else ''

        new_user_id = users_df['user_id'].max() + 1 if not users_df.empty else 1

        new_user = pd.DataFrame([{
            'user_id': new_user_id,
            'username': username,
            'password': password,
            'first_name': first_name,
            'last_name': last_name,
            'age': age,
            'gender': gender,
            'favorite_artists': favorite_artists_str,
            'recommended_artists': ''
        }])

        users_df = pd.concat([users_df, new_user], ignore_index=True)
        self.save_users_df(users_df)

        # Update recommended artists
        self.update_recommended_artists(new_user_id, favorite_artists or [])

        return (True, new_user_id)

    def get_favorite_artists(self, user_id):
        """
        Get a user's favorite artists.

        Args:
            user_id (int): The user ID

        Returns:
            list: The user's favorite artists
        """
        user = self.get_user_by_id(user_id)

        if user is None:
            return []

        fav_str = user.get('favorite_artists', '')

        if pd.isna(fav_str) or not fav_str:
            return []

        return [a.strip() for a in fav_str.split(',') if a.strip()]

    def update_favorite_artists(self, user_id, favorite_artists):
        """
        Update a user's favorite artists.

        Args:
            user_id (int): The user ID
            favorite_artists (list): List of favorite artists

        Returns:
            bool: True if successful, False otherwise
        """
        users_df = self.get_users_df()
        user_rows = users_df['user_id'] == user_id

        if not any(user_rows):
            return False

        favorite_artists_str = ', '.join(favorite_artists) if favorite_artists else ''
        users_df.loc[user_rows, 'favorite_artists'] = favorite_artists_str
        self.save_users_df(users_df)

        # Update recommended artists based on new favorites
        self.update_recommended_artists(user_id, favorite_artists)

        return True

    def update_recommended_artists(self, user_id, favorite_artists):
        """
        Update recommended artists based on favorites.
        This is a simple implementation - in a real app, this would use
        a more sophisticated recommendation algorithm.

        Args:
            user_id (int): The user ID
            favorite_artists (list): List of favorite artists

        Returns:
            bool: True if successful, False otherwise
        """
        # Simple implementation: just store the favorite artists as recommendations
        # In a real app, this would calculate actual recommendations
        if not favorite_artists:
            return False

        users_df = self.get_users_df()
        user_rows = users_df['user_id'] == user_id

        if not any(user_rows):
            return False

        users_df.loc[user_rows, 'recommended_artists'] = ', '.join(favorite_artists)
        self.save_users_df(users_df)

        return True

Overwriting /content/drive/MyDrive/Colab Notebooks/project/final/services/user_manager.py
